In [8]:
import torchwordemb
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim


## running Models and dataloader 

In [2]:
%run ./models.ipynb
%run ./data_loader.ipynb


/scratch/borna/apps/venv/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


tensor([[ 0.0013,  0.0015, -0.0013,  ..., -0.0004, -0.0006,  0.0010],
        [-0.0237, -0.0711,  0.0030,  ..., -0.0499,  0.0782,  0.0468],
        [-0.0555,  0.0136, -0.0149,  ..., -0.0237,  0.1307, -0.0041],
        ...,
        [-0.0521,  0.1869, -0.2185,  ...,  0.2384, -0.6327, -0.2994],
        [ 0.0019, -0.0814,  0.0135,  ...,  0.0355, -0.1582,  0.0095],
        [ 0.3283, -0.1053, -0.3926,  ..., -0.1166, -0.2159, -0.1853]])
torch.Size([1, 20, 300])
torch.Size([1, 2048])


## Parameter Definition

In [5]:
weight_decay = 5e-4
num_hidden = 8
dropout_rate = 0.6
init_type = 'uniform'
lr = 5e-3
lr_decay_epoch = 5000
use_cude = True

## Forming nodes and edges of graph

In [ ]:
def build_graph(node_lengths):
    '''
    :param graphs: a list of graphs (each graph is a dictionary)
    'e.g. { 'num_nodes': 3,
            'edges':{0: [(0,1), (1, 0)]} } edge_type: edges
    '
    :return: a list of adjacent matrices and hidden mask
    '''  
         
    for node_length in node_lengths:
        
        edge_type = 0 #edge_type 0 corresponds to edges between the ingredients
        nodes = list(range(0,node_length))
        
        graph_temp = {}
        graph_temp['num_nodes'] = node_length
        graph_temp['edges'] = {edge_type: [(x,y) for x in nodes for y in nodes]}

In [ ]:
def build_adj_matrix(graphs, n_edges = 1):
        '''
        :param graphs: a list of graphs (each graph is a dictionary)
        'e.g. { 'num_nodes': 3,
                'edges':{0: [(0,1), (1, 0)]} } edge_type: edges
        '
        :return: a list of adjacent matrices and hidden mask
        '''

        num_graphs = len(graphs)
        #max_num_nodes = max([G['num_nodes'] for G in graphs])
        
        # adjacency matrices with max size as max number of nodes across all graphs
        batch_adj_matrices = to_var(torch.zeros(n_edges,num_graphs, max_num_nodes, max_num_nodes))
        # mask - BS x MaxNodes (to kill hidden states for not existing nodes)
        batch_mask = np.ones((n_edges, num_graphs, max_num_nodes, 1), dtype='float32')  # third dim present to apply mask easily :)
        for g, graph in enumerate(graphs):
            num_nodes = graph['num_nodes']
            edges = graph['edges']
            for e_type, links in edges.items():
                if e_type == 0:
                    for src, dst, dist in links:
                        batch_adj_matrices[e_type,g, dst, src] = dist
                else if e_type == 1:
                    for src, dst in links:
                        batch_adj_matrices[e_type, g, dst, src] = 1

            # set things in mask to 0
            batch_mask[:,g, num_nodes:] = 0
        return batch_adj_matrices, to_var(torch.from_numpy(batch_mask))

## GCD train for one epoch

In [ ]:
def GCDtrain(train_loader, adj,model, criterion, optimizer, epoch):
    
    # switch to train mode
    model.train()
    
    optimizer.lr = lr_scheduler(epoch)
    optimizer.zero_grad()
    
    for i, (img, instrs, ingrs, category, target) in enumerate(training_generator): 
        
        
        _, vec = torchwordemb.load_word2vec_bin('../data/vocab.bin')
        ingrs_embs = nn.Embedding(vec.size(0), 300, padding_idx=0)
        
        visual_emb, recipe_emb = model(ingrs, adj)
        
        loss_gcdtrain = criterion(visual_emb, recipe_emb, target)
        
        loss_gcdtrain.backward()
        optimizer.step()

In [6]:
def main():

    batch_size = 1

    params = {'batch_size': batch_size, 'shuffle':True, 'pin_memory':True}

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])

    transform = transforms.Compose([
                transforms.Scale(256), # rescale the image keeping the original aspect ratio
                transforms.CenterCrop(256), # we get only the center of that rescaled
                transforms.RandomCrop(224), # random crop within the center crop 
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize
                ])


    training_set = Loader(data_path='../data', partition = 'train', img_path='../data/' , transform=transform)
    training_generator = data.DataLoader(training_set, **params)


    model = GCN(
            nfeat = 300,
            nhid = num_hidden,
            dropout = dropout_rate,
            init = init_type
            )
 
    optimizer = optim.SGD(
            model.parameters(),
            lr = lr,
            weight_decay = weight_decay,
            momentum = 0.9
    )
    
        
    if use_cude:
        model.cuda()
        cosine_crit = nn.CosineEmbeddingLoss(0.1).cuda()
    else:
        cosine_crit = nn.CosineEmbeddingLoss(0.1)
        
    criterion = cosine_crit
      
        
    def lr_scheduler(epoch):
        return lr * (0.5 ** (epoch / lr_decay_epoch))

    epochs = 1
# run epochs
for epoch in range(epochs):
   
    # train for one epoch
    adj, to_var =build_adj_matrix(model, n_edges = 1)
    train(training_generator, adj,model,criterion, optimizer, epoch)

   
if __name__ == '__main__':
    main()

| Uniform Initialization
| Uniform Initialization
